In [1]:
import csv, os, re

path = os.path.expanduser("~/Downloads/uj2eoqfttsdadhbb.csv")

# ISIN validation (ISO 6166 + Luhn)
_ISIN_RE = re.compile(r"^[A-Z]{2}[A-Z0-9]{9}[0-9]$")
def is_valid_isin(code: str) -> bool:
    s = (code or "").strip().upper().replace(" ", "")
    if not _ISIN_RE.match(s):
        return False
    # Expand letters A=10 ... Z=35 then apply Luhn over the full string (incl. check digit)
    expanded = "".join(str(ord(c) - 55) if c.isalpha() else c for c in s)
    total = 0
    for i, ch in enumerate(expanded[::-1]):  # right-to-left
        d = int(ch)
        if i % 2 == 1:  # double every second digit
            d *= 2
            if d > 9:
                d -= 9
        total += d
    return total % 10 == 0

unique_isins = set()

with open(path, newline="", encoding="utf-8", errors="ignore") as f:
    reader = csv.reader(f)
    headers = next(reader)
    # Try to find a column literally named "isin" (any case/spacing)
    norm = [re.sub(r"\W+", "", (h or "")).lower() for h in headers]
    isin_idx = norm.index("isin") if "isin" in norm else None

    for row in reader:
        cells = []
        if isin_idx is not None and isin_idx < len(row):
            cells = [row[isin_idx]]            # fast path: dedicated ISIN column
        else:
            cells = row                        # fallback: scan all cells per row

        for cell in cells:
            if is_valid_isin(cell):
                unique_isins.add(cell.strip().upper())

print(f"Found {len(unique_isins)} unique ISINs")
# If you want to save them:
out_path = os.path.expanduser("~/Downloads/unique_isins.txt")
with open(out_path, "w", encoding="utf-8") as out:
    out.write("\n".join(sorted(unique_isins)))
print(f"Wrote to: {out_path}")

KeyboardInterrupt: 